In [236]:
import pandas as pd
import numpy as np
import math
import TreeNode

# class TreeNode:
#     def __init__(self, label=None, attributes=None, children=None):
#         self.value = label  # value of the node
#         self.attributes = attributes if attributes is not None else {}  # attributes in dict
#         self.children = children or {}  # dict of child nodes
    
#     def add_child(self, child_value, child_node):
#         self.children[child_value] = child_node

#     def __str__(self, level=0):
#         prefix = "  " * level
#         result = prefix + f"Attribute: {self.attributes}, Label: {self.value}\n"
    
#         for child_value, child_node in self.children.items():
#             result += prefix + f"Value: {child_value}\n"
#             result += child_node.__str__(level + 1)
    
#         return result

# class TreeNode: 
#     def __init__(self, label=None, attributes=None, children=None):
#         self.value = label  # value of the node
#         self.attributes = attributes
#         self.children = children or []  # dict of child nodes
    
#     def __str__(self, level=0):
#         prefix = "  " * level
#         result = prefix + f"Attribute: {self.attributes}, Label: {self.value}\n"
#         for child in self.children:
#             result += prefix + f"Child:\n"
#             result += child.__str__(level + 1)
#         return result
#     # def __str__(self):
#     #     return str(self.value)

#     def add_child(self, child_node):
#         self.children.append(child_node)


def calculate_entropy(feature_value_data, class_list):
    """Function to calculate entropy"""
    # feature_value_data: Subdataset with feature value data
    # class_list: the unique class list in target variable
    feature_total_count = feature_value_data.shape[0]
    label_list = feature_value_data.iloc[:, -1].tolist()
    entropy = 0
    
    for class_label in class_list:
        class_count = label_list.count(class_label)
        class_probability = class_count / feature_total_count
        if class_probability > 0:
            class_entropy = -class_probability * math.log2(class_probability)
            entropy += class_entropy

    return entropy

def calculate_majority_error(feature_value_data, class_list):
    """Function to calculate majority error"""
    feature_total_count = feature_value_data.shape[0]
    label_list = feature_value_data.iloc[:, -1].tolist()
    majority_error = 0
    
    majority_class_count = max([label_list.count(c) for c in class_list])
    majority_error = 1 - (majority_class_count / feature_total_count)
    
    return majority_error

def calculate_gini_index(feature_value_data, class_list):
    """Function to calculate gini index"""
    feature_total_count = feature_value_data.shape[0]
    label_list = feature_value_data.iloc[:, -1].tolist()
    gini_index = 0
    
    for class_label in class_list:
        class_count = label_list.count(class_label)
        class_probability = class_count / feature_total_count
        gini_index += class_probability * (1 - class_probability)
    
    return gini_index


def calculate_info_gain(feature_name, data, class_list, purity_measurement):
    """Function to calculate information gain for a specfici feature/attribute"""
    # purity_measurement should be one of entropy, majority_error, gini
    feature_value_list = data[feature_name].unique()
    total_row = data.shape[0]
    feature_info = 0.0
    
    for feature_value in feature_value_list:
        feature_value_data = data[data[feature_name] == feature_value] #filtering rows with that feature_value
        feature_value_count = feature_value_data.shape[0]
        if purity_measurement == 'entropy':
            total_entropy = calculate_entropy(data, class_list)
            feature_value_entropy = calculate_entropy(feature_value_data, class_list) #calculcating entropy for the feature value
            feature_value_probability = feature_value_count/total_row
            feature_info += feature_value_probability * feature_value_entropy #calculating information of the feature value
            
        elif purity_measurement == 'majority_error':
            total_entropy = calculate_majority_error(data, class_list)
            feature_value_entropy = calculate_majority_error(feature_value_data, class_list) #calculcating entropy for the feature value
            feature_value_probability = feature_value_count/total_row
            feature_info += feature_value_probability * feature_value_entropy #calculating information of the feature value
        elif purity_measurement == 'gini':
            total_entropy = calculate_gini_index(data, class_list)
            feature_value_entropy = calculate_gini_index(feature_value_data, class_list) #calculcating entropy for the feature value
            feature_value_probability = feature_value_count/total_row
            feature_info += feature_value_probability * feature_value_entropy #calculating information of the feature value
        else: 
            print("No valid purity_measurement has been input. We only support entropy, majority_error and gini. Please further check! ")
            raise ValueError
    return total_entropy - feature_info


def find_best_attribute(data, attributes, class_list, purity_measurement):
    print("starting finding the best feature")
    # Get the feature columns (all columns except the label column)
    print("data count for input df: ", data.shape[0])
    feature_list = data.columns[:-1].tolist()
    print("feature list in find best_attribute", feature_list)
    max_info_gain = -1
    max_info_feature = None

    for attribute in attributes:  #for each feature in the dataset
        feature_info_gain = calculate_info_gain(attribute, data, class_list, purity_measurement)
        print("For {} the information gain is {}".format(attribute, feature_info_gain))
        if max_info_gain < feature_info_gain: #selecting feature name with highest information gain
            max_info_gain = feature_info_gain
            max_info_feature = attribute
            
    return max_info_feature

# Find most common label
def find_most_common_label(df):
    labels = df.iloc[:, -1].tolist()
    label_counts = count_elements(labels)
    most_common_label = max(label_counts, key=lambda k: label_counts[k])
    return most_common_label

def count_elements(lst):
    element_counts = {}
    for element in lst:
        if element in element_counts:
            element_counts[element] += 1
        else:
            element_counts[element] = 1
    return element_counts

# def ID3(S, Attributes, Label, max_depth=None, purity_measurement=None):
#     """
#     S is a dataframe of the dataset
#     Attributes are the value of the attributes
#     Label are the list of the target variable for the datase
#     Default purity measurement for ID3 is entropy
#     """ 
    
#     if not purity_measurement:
#         purity_measurement = 'entropy'  # Default purity measurement
#     print('Starting implementing ID3 with purity_measurement {}'.format(purity_measurement))

#     # Check if leaf mode with the same label
#     unique_labels = set(Label)
#     class_list = list(unique_labels)
#     if len(unique_labels) == 1:
#         print('unique labels == 1')
#         return TreeNode(label=unique_labels.pop())
#     # Check if attribute is empty
#     elif not Attributes:
#         most_common_label = find_most_common_label(S)
#         print("attibute is empty, find most common label: ", most_common_label)
#         return TreeNode(label=most_common_label)
#     else:
#         # Create a Root Node for tree
#         root = TreeNode()
#         print("------starting creating root node-------")
#         # Choose the best attribute A to split S
#         # find_most_informative_feature(data, class_list, purity_measurement):
#         best_attribute = find_best_attribute(S, class_list, purity_measurement)
#         print("best attribute: ", best_attribute)
#         root.attributes = best_attribute 
#         attribute_values = S[best_attribute].unique().tolist()
#         remaining_attributes = [attr for attr in Attributes if attr != best_attribute]
#         print("remaining_attributes: ", remaining_attributes)
        
#         # deal with the remaining attributes for subset Sv, according to A=V
#         for value in attribute_values:
#             print("value in attribute values:", value)
#             child_node = TreeNode()
#             child_node.attributes = value  
#             root.add_child(child_node) 
#             print("Check current root: ", root)

#             Sv = S[S[best_attribute] == value]
#             remaining_label = Sv.iloc[:,-1].tolist()
#             # If Sv is empty, add leaf node with the most common value of label in S
#             if Sv.empty:
#                 most_common_label = find_most_common_label(Sv)
#                 root.children.value = TreeNode(label=most_common_label)
#                 print("return root when Sv is empty",root)
#             else:
#                 return ID3(Sv, remaining_attributes, remaining_label, purity_measurement)
#     return root

In [237]:
# def ID3(S, Attributes, Label, max_depth=None, purity_measurement=None, root=None):
#     """
#     S is a dataframe of the dataset
#     Attributes are the value of the attributes
#     Label are the list of the target variable for the dataset
#     Default purity measurement for ID3 is entropy
#     """ 
    
#     if not purity_measurement:
#         purity_measurement = 'entropy'  # Default purity measurement
#     print('Starting implementing ID3 with purity_measurement {}'.format(purity_measurement))

#     # Check if leaf mode with the same label
#     unique_labels = set(Label)
#     class_list = list(unique_labels)
#     print("class_list check: ", class_list)
#     if len(unique_labels) == 1:
#         print('unique labels == 1')
#         if root is None:
#             return TreeNode(label=unique_labels.pop())
#         else:
#             root.label = unique_labels.pop()
#             return root
#     # Check if attribute is empty
#     elif not Attributes:
#         most_common_label = find_most_common_label(S)
#         print("attribute is empty, find most common label: ", most_common_label)
#         if root is None:
#             return TreeNode(label=most_common_label)
#         else:
#             root.label = most_common_label
#             return root
#     else:
#         if root is None:
#             print("----root is None, first time to create tree node")
#             root = TreeNode()
#         # Choose the best attribute A to split S
#         # find_most_informative_feature(data, class_list, purity_measurement):
#         print("class_list check: ", class_list)
#         best_attribute = find_best_attribute(S, class_list, purity_measurement)
#         print("best attribute: ", best_attribute)
#         root.attributes = best_attribute 
#         attribute_values = S[best_attribute].unique().tolist()
#         remaining_attributes = [attr for attr in Attributes if attr != best_attribute]
#         print("remaining_attributes: ", remaining_attributes)
#         # deal with the remaining attributes for subset Sv, according to A=V
#         for value in attribute_values:
#             print("value in attribute values:", value)
#             child_node = TreeNode()
#             child_node.attributes = value  
#             root.add_child(child_node) 
#             print("Check current root: ", root)

#             Sv = S[S[best_attribute] == value]
#             remaining_label = Sv.iloc[:,-1].tolist()
#             # If Sv is empty, add leaf node with the most common value of label in S
#             if Sv.empty:
#                 most_common_label = find_most_common_label(Sv)
#                 child_node.label = most_common_label
#                 print("return root when Sv is empty",root)
#             else:
#                 ID3(Sv, remaining_attributes, remaining_label, purity_measurement, root=root)
#     return root


In [352]:
# test2 -!! this one work!!

class TreeNode: 
    def __init__(self, label=None, attributes=None, children=None):
        self.label = label  # value of the node
        self.attributes = attributes
        self.children = children or []  # dict of child nodes
    
    def __str__(self, level=0):
        prefix = "  " * level
        result = prefix + f"Attribute: {self.attributes}, Label: {self.label}\n"
        for child in self.children:
            result += prefix + f"Child:\n"
            result += child.__str__(level + 1)
        return result
    # def __str__(self):
    #     return str(self.value)

    def add_child(self, child_node):
        self.children.append(child_node)
        
def ID3(S, Attributes, max_depth, purity_measurement=None, root=None):
    if not purity_measurement:
        purity_measurement = 'entropy'  # Default purity measurement
    # Check if leaf mode with the same label
    Label = S.iloc[:,-1].tolist()
    unique_labels = set(Label)
    class_list = list(unique_labels)  
    if len(unique_labels) == 1:
        get_unique_label = next(iter(unique_labels))
        if root is None:
            return TreeNode(label=get_unique_label)
        else:
            root.label = get_unique_label
            return root
    
    # Check if attribute is empty
    elif not Attributes or max_depth==0:
        most_common_label = find_most_common_label(S)
        if root is None:
            return TreeNode(label=most_common_label)
        else:
            root.label = most_common_label
            return root
    
    else:
        if root is None:
            root = TreeNode()
            print("------Root is none, first time creating root node-------")
        print("------starting creating root node-------")
        # Choose the best attribute A to split S
        print("check current root:", root)
        best_attribute = find_best_attribute(S, Attributes, class_list, purity_measurement)
        print("best attribute: ", best_attribute)
        root.attributes = best_attribute 
        
        # Create a new list for remaining attributes
        remaining_attributes = [attr for attr in Attributes if attr != best_attribute]
        print("remaining_attributes: ", remaining_attributes)  
        # Deal with the remaining attributes for subset Sv, according to A=V
        attribute_values = S[best_attribute].unique().tolist()
        
        for value in attribute_values:
            print("value in attribute values:", value)
            child_node = TreeNode()
            child_node.attributes = value  
            root.add_child(child_node) 
            print("Check current root: ", root)

            Sv = S[S[best_attribute] == value]
            
            # If Sv is empty, add leaf node with the most common value of label in S
            if Sv.empty:
                most_common_label = find_most_common_label(Sv)
                child_node.label = most_common_label
                
                print("check child node: ", child_node)
            else:
                print("starting create sub tree")
                ID3(Sv, remaining_attributes, max_depth-1, purity_measurement, root=child_node)
    return root


In [356]:
# Sample Usage
S = [
    {"Outlook": "Sunny", "Temperature": "Hot", "Humidity": "High", "Wind": "Weak", "Play": "-"},
    {"Outlook": "Sunny", "Temperature": "Hot", "Humidity": "High", "Wind": "Strong", "Play": "-"},
    {"Outlook": "Overcast", "Temperature": "Hot", "Humidity": "High", "Wind": "Weak", "Play": "+"},
    {"Outlook": "Rain", "Temperature": "Mild", "Humidity": "High", "Wind": "Weak", "Play": "+"},
    {"Outlook": "Rain", "Temperature": "Cool", "Humidity": "Normal", "Wind": "Weak", "Play": "+"},
    {"Outlook": "Rain", "Temperature": "Cool", "Humidity": "Normal", "Wind": "Strong", "Play": "-"},
    {"Outlook": "Overcast", "Temperature": "Cool", "Humidity": "Normal", "Wind": "Strong", "Play": "+"},
    {"Outlook": "Sunny", "Temperature": "Mild", "Humidity": "High", "Wind": "Weak", "Play": "-"},
    {"Outlook": "Sunny", "Temperature": "Cool", "Humidity": "Normal", "Wind": "Weak", "Play": "+"},
    {"Outlook": "Rain", "Temperature": "Mild", "Humidity": "Normal", "Wind": "Weak", "Play": "+"},
    {"Outlook": "Sunny", "Temperature": "Mild", "Humidity": "Normal", "Wind": "Strong", "Play": "+"},
    {"Outlook": "Overcast", "Temperature": "Mild", "Humidity": "High", "Wind": "Strong", "Play": "+"},
    {"Outlook": "Overcast", "Temperature": "Hot", "Humidity": "Normal", "Wind": "Weak", "Play": "+"},
    {"Outlook": "Rain", "Temperature": "Mild", "Humidity": "High", "Wind": "Weak", "Play": "-"},
]

df_S = pd.DataFrame(S)

Attributes = ["Outlook", "Temperature", "Humidity", "Wind"]
Label = ["-", "-", "+", "+", "+", "-", "+", "-", "+", "+", "+", "+", "+", "-"]
Target_var = 'Play'


# root = ID3(df_S, Attributes, Label, purity_measurement='entropy')  # You can change purity_measurement as needed
root = ID3(df_S, Attributes, max_depth=2, purity_measurement='entropy')


------Root is none, first time creating root node-------
------starting creating root node-------
check current root: Attribute: None, Label: None

starting finding the best feature
data count for input df:  14
feature list in find best_attribute ['Outlook', 'Temperature', 'Humidity', 'Wind']
For Outlook the information gain is 0.24674981977443933
For Temperature the information gain is 0.02922256565895487
For Humidity the information gain is 0.15183550136234159
For Wind the information gain is 0.003184853044648994
best attribute:  Outlook
remaining_attributes:  ['Temperature', 'Humidity', 'Wind']
value in attribute values: Sunny
Check current root:  Attribute: Outlook, Label: None
Child:
  Attribute: Sunny, Label: None

starting create sub tree
------starting creating root node-------
check current root: Attribute: Sunny, Label: None

starting finding the best feature
data count for input df:  5
feature list in find best_attribute ['Outlook', 'Temperature', 'Humidity', 'Wind']
For Tem

In [357]:
print(root)

Attribute: Outlook, Label: None
Child:
  Attribute: Humidity, Label: None
  Child:
    Attribute: High, Label: -
  Child:
    Attribute: Normal, Label: +
Child:
  Attribute: Overcast, Label: +
Child:
  Attribute: Wind, Label: None
  Child:
    Attribute: Weak, Label: +
  Child:
    Attribute: Strong, Label: -



In [334]:
print(root)

Attribute: Outlook, Label: None
Child:
  Attribute: Humidity, Label: None
  Child:
    Attribute: High, Label: -
  Child:
    Attribute: Normal, Label: +
Child:
  Attribute: Overcast, Label: +
Child:
  Attribute: Wind, Label: None
  Child:
    Attribute: Weak, Label: +
  Child:
    Attribute: Strong, Label: -



In [310]:
# Sample Usage
S_test = [
    {"Outlook": "Sunny", "Temperature": "Hot", "Humidity": "High", "Wind": "Weak", "Play": "+"},
    {"Outlook": "Sunny", "Temperature": "Hot", "Humidity": "High", "Wind": "Strong", "Play": "-"},
    {"Outlook": "Overcast", "Temperature": "Hot", "Humidity": "High", "Wind": "Weak", "Play": "+"},
    {"Outlook": "Rain", "Temperature": "Mild", "Humidity": "High", "Wind": "Weak", "Play": "+"},
    {"Outlook": "Rain", "Temperature": "Cool", "Humidity": "Normal", "Wind": "Weak", "Play": "+"},
    {"Outlook": "Rain", "Temperature": "Cool", "Humidity": "Normal", "Wind": "Strong", "Play": "-"},
    {"Outlook": "Overcast", "Temperature": "Cool", "Humidity": "Normal", "Wind": "Strong", "Play": "+"},
    {"Outlook": "Sunny", "Temperature": "Mild", "Humidity": "High", "Wind": "Weak", "Play": "-"},
    {"Outlook": "Sunny", "Temperature": "Cool", "Humidity": "Normal", "Wind": "Weak", "Play": "+"},
    {"Outlook": "Rain", "Temperature": "Mild", "Humidity": "Normal", "Wind": "Weak", "Play": "+"},
    {"Outlook": "Sunny", "Temperature": "Mild", "Humidity": "Normal", "Wind": "Strong", "Play": "-"},
    {"Outlook": "Overcast", "Temperature": "Mild", "Humidity": "High", "Wind": "Strong", "Play": "+"},
    {"Outlook": "Overcast", "Temperature": "Hot", "Humidity": "Normal", "Wind": "Weak", "Play": "+"},
    {"Outlook": "Rain", "Temperature": "Mild", "Humidity": "High", "Wind": "Weak", "Play": "-"},
]

df_test = pd.DataFrame(S_test)

In [299]:
def predict(tree, test_data):

    while tree.children:
        current_attribute = tree.attributes
        attribute_value = test_data.get(current_attribute)

        matched_child = None
        for child in tree.children:
            if child.attributes == attribute_value:
                matched_child = child
                break

        if matched_child:
            tree = matched_child
        else:
            return tree.label
        
    return tree.label


# predictions = []
# for index, row in df_test.iterrows():
#     prediction = predict(root, row)  
#     predictions.append(prediction) 


In [311]:
def make_predictions(tree_root, test_data):
    predictions = []
    for index, row in test_data.iterrows():
        prediction = predict(tree_root, row)  
        predictions.append(prediction)
    return predictions


In [312]:

predictions = make_predictions(root, df_test)
print(predictions)

['-', '-', '+', '+', '+', '+', '+', '-', '-', '+', '-', '+', '+', '+']


In [314]:
true_labels = df_test.iloc[:, -1].tolist()
print(true_labels)

['+', '-', '+', '+', '+', '-', '+', '-', '+', '+', '-', '+', '+', '-']


In [315]:
error_rate = calculate_error_rate(predictions, true_labels)
print(error_rate)

0.2857142857142857


In [319]:
# Sample Usage
# Sample Usage
S = [
    {"Outlook": "Sunny", "Temperature": "Hot", "Humidity": "High", "Wind": "Weak", "Play": "-"},
    {"Outlook": "Sunny", "Temperature": "Hot", "Humidity": "High", "Wind": "Strong", "Play": "-"},
    {"Outlook": "Overcast", "Temperature": "Hot", "Humidity": "High", "Wind": "Weak", "Play": "+"},
    {"Outlook": "Rain", "Temperature": "Mild", "Humidity": "High", "Wind": "Weak", "Play": "+"},
    {"Outlook": "Rain", "Temperature": "Cool", "Humidity": "Normal", "Wind": "Weak", "Play": "+"},
    {"Outlook": "Rain", "Temperature": "Cool", "Humidity": "Normal", "Wind": "Strong", "Play": "-"},
    {"Outlook": "Overcast", "Temperature": "Cool", "Humidity": "Normal", "Wind": "Strong", "Play": "+"},
    {"Outlook": "Sunny", "Temperature": "Mild", "Humidity": "High", "Wind": "Weak", "Play": "-"},
    {"Outlook": "Sunny", "Temperature": "Cool", "Humidity": "Normal", "Wind": "Weak", "Play": "+"},
    {"Outlook": "Rain", "Temperature": "Mild", "Humidity": "Normal", "Wind": "Weak", "Play": "+"},
    {"Outlook": "Sunny", "Temperature": "Mild", "Humidity": "Normal", "Wind": "Strong", "Play": "+"},
    {"Outlook": "Overcast", "Temperature": "Mild", "Humidity": "High", "Wind": "Strong", "Play": "+"},
    {"Outlook": "Overcast", "Temperature": "Hot", "Humidity": "Normal", "Wind": "Weak", "Play": "+"},
    {"Outlook": "Rain", "Temperature": "Mild", "Humidity": "High", "Wind": "Weak", "Play": "-"},
]

df_train = pd.DataFrame(S)

Attributes = ["Outlook", "Temperature", "Humidity", "Wind"]

# root = ID3(df_S, Attributes, Label, purity_measurement='entropy')  # You can change purity_measurement as needed
# root = ID3(df_S, Attributes, max_depth=1, purity_measurement='entropy')

S_test = [
    {"Outlook": "Sunny", "Temperature": "Hot", "Humidity": "High", "Wind": "Weak", "Play": "+"},
    {"Outlook": "Sunny", "Temperature": "Hot", "Humidity": "High", "Wind": "Strong", "Play": "-"},
    {"Outlook": "Overcast", "Temperature": "Hot", "Humidity": "High", "Wind": "Weak", "Play": "+"},
    {"Outlook": "Rain", "Temperature": "Mild", "Humidity": "High", "Wind": "Weak", "Play": "+"},
    {"Outlook": "Rain", "Temperature": "Cool", "Humidity": "Normal", "Wind": "Weak", "Play": "+"},
    {"Outlook": "Rain", "Temperature": "Cool", "Humidity": "Normal", "Wind": "Strong", "Play": "-"},
    {"Outlook": "Overcast", "Temperature": "Cool", "Humidity": "Normal", "Wind": "Strong", "Play": "+"},
    {"Outlook": "Sunny", "Temperature": "Mild", "Humidity": "High", "Wind": "Weak", "Play": "-"},
    {"Outlook": "Sunny", "Temperature": "Cool", "Humidity": "Normal", "Wind": "Weak", "Play": "+"},
    {"Outlook": "Rain", "Temperature": "Mild", "Humidity": "Normal", "Wind": "Weak", "Play": "+"},
    {"Outlook": "Sunny", "Temperature": "Mild", "Humidity": "Normal", "Wind": "Strong", "Play": "-"},
    {"Outlook": "Overcast", "Temperature": "Mild", "Humidity": "High", "Wind": "Strong", "Play": "+"},
    {"Outlook": "Overcast", "Temperature": "Hot", "Humidity": "Normal", "Wind": "Weak", "Play": "+"},
    {"Outlook": "Rain", "Temperature": "Mild", "Humidity": "High", "Wind": "Weak", "Play": "-"},
]

df_test = pd.DataFrame(S_test)

In [317]:
# run tree and make prediction and cal error
root = ID3(df_train, Attributes, max_depth=1, purity_measurement='entropy')
predictions = make_predictions(root, df_test)
true_labels = df_test.iloc[:, -1].tolist()
error_rate = calculate_error_rate(predictions, true_labels)

Starting implementing ID3 with purity_measurement entropy
class list in this iteration ['+', '-']
------Root is none, first time creating root node-------
------starting creating root node-------
starting finding the best feature
data count for input df:  14
feature list in find best_attribute ['Outlook', 'Temperature', 'Humidity', 'Wind']
For Outlook the information gain is 0.24674981977443933
For Temperature the information gain is 0.02922256565895487
For Humidity the information gain is 0.15183550136234159
For Wind the information gain is 0.003184853044648994
best attribute:  Outlook
remaining_attributes:  ['Temperature', 'Humidity', 'Wind']
value in attribute values: Sunny
Check current root:  Attribute: Outlook, Label: None
Child:
  Attribute: Sunny, Label: None

Starting implementing ID3 with purity_measurement entropy
class list in this iteration ['+', '-']
attribute is empty, find most common label:  -
value in attribute values: Overcast
Check current root:  Attribute: Outlook,

In [318]:
print(error_rate)

0.2857142857142857


In [307]:
max_depths = range(1, 7)
# column_names = ['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety', 'label']
# df_train, attributes = read_data("Data/car-4/train.csv", column_names)
# df_test, attributes = read_data("Data/car-4/test.csv", column_names)

for max_depth in max_depths:
    print(f"Testing with max_depth = {max_depth}")
    
    root = ID3(df_S, Attributes, max_depth=max_depth, purity_measurement='entropy')
    
    testing_predictions = make_predictions(root, df_test)
    testing_true_labels = df_test.iloc[:, -1].tolist()
    testing_error_rate = calculate_error_rate(testing_predictions, testing_true_labels)

    print(f"Testing Error Rate with max_depth {max_depth}: {testing_error_rate}")

Testing with max_depth = 1
Starting implementing ID3 with purity_measurement entropy
class list in this iteration ['+', '-']
------Root is none, first time creating root node-------
------starting creating root node-------
starting finding the best feature
data count for input df:  14
feature list in find best_attribute ['Outlook', 'Temperature', 'Humidity', 'Wind']
For Outlook the information gain is 0.24674981977443933
For Temperature the information gain is 0.02922256565895487
For Humidity the information gain is 0.15183550136234159
For Wind the information gain is 0.003184853044648994
best attribute:  Outlook
remaining_attributes:  ['Temperature', 'Humidity', 'Wind']
value in attribute values: Sunny
Check current root:  Attribute: Outlook, Label: None
Child:
  Attribute: Sunny, Label: None

Starting implementing ID3 with purity_measurement entropy
class list in this iteration ['+', '-']
attribute is empty, find most common label:  -
value in attribute values: Overcast
Check current

In [254]:
true_labels = df_test.iloc[:, -1].tolist()

In [303]:
def read_data(file_path, column_names):
    # Read the CSV file with the specified column names
    df = pd.read_csv(file_path, names=column_names)
    # Extract the list of attributes from the DataFrame's columns and drop the last column (label)
    attributes = df.columns.tolist()[:-1]
    return df, attributes


def calculate_error_rate(predictions, true_labels):
    """
    predictions: list of prediction labels using ID3
    true_labels: real labels
    error_rate
    """
    if len(predictions) != len(true_labels):
        raise ValueError("Number of predictions and true label do not match")

    incorrect_predictions = 0
    total_samples = len(predictions)

    for i in range(total_samples):
        if predictions[i] != true_labels[i]:
            incorrect_predictions += 1

    error_rate = incorrect_predictions / total_samples
    return error_rate

In [304]:
max_depths = range(1, 7)
column_names = ['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety', 'label']
df_train, attributes = read_data("Data/car-4/train.csv", column_names)
df_test, attributes = read_data("Data/car-4/test.csv", column_names)

for max_depth in max_depths:
    print(f"Testing with max_depth = {max_depth}")
    
    root = ID3(df_train, attributes, max_depth=max_depth, purity_measurement='entropy')
    
    testing_predictions = make_predictions(root, df_test)
    testing_true_labels = df_test.iloc[:, -1].tolist()
    testing_error_rate = calculate_error_rate(testing_predictions, testing_true_labels)

    print(f"Testing Error Rate with max_depth {max_depth}: {testing_error_rate}")


Testing with max_depth = 1
Starting implementing ID3 with purity_measurement entropy
class list in this iteration ['good', 'vgood', 'acc', 'unacc']
------Root is none, first time creating root node-------
------starting creating root node-------
starting finding the best feature
data count for input df:  1000
feature list in find best_attribute ['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety']
For buying the information gain is 0.10152470712485662
For maint the information gain is 0.07741985577459665
For doors the information gain is 0.006726514230977809
For persons the information gain is 0.22441128678577127
For lug_boot the information gain is 0.03688725199484155
For safety the information gain is 0.25822501448993573
best attribute:  safety
remaining_attributes:  ['buying', 'maint', 'doors', 'persons', 'lug_boot']
value in attribute values: med
Check current root:  Attribute: safety, Label: None
Child:
  Attribute: med, Label: None

Starting implementing ID3 with purity_m

In [255]:
def calculate_error_rate(predictions, true_labels):
    """
    predictions: list of prediction labels using ID3
    true_labels: real labels
    error_rate
    """
    if len(predictions) != len(true_labels):
        raise ValueError("Number of predictions and true label do not match")

    incorrect_predictions = 0
    total_samples = len(predictions)

    for i in range(total_samples):
        if predictions[i] != true_labels[i]:
            incorrect_predictions += 1

    error_rate = incorrect_predictions / total_samples
    return error_rate

In [256]:
# Calculate testing error
testing_predictions = predictions
testing_true_labels = df_test.iloc[:, -1].tolist()

testing_error_rate = calculate_error_rate(testing_predictions, testing_true_labels)
print("Testing Error Rate:", testing_error_rate)

Testing Error Rate: 0.2857142857142857


In [259]:
# Read data
df_car_train = pd.read_csv("Data/car-4/train.csv")

In [260]:
df_car_train.head()

,low,vhigh,4,4.1,big,med,acc
0,low,high,5more,4,med,high,vgood
1,vhigh,med,2,2,big,high,unacc
2,high,high,2,2,small,high,unacc
3,vhigh,low,3,2,big,low,unacc
4,high,high,3,4,med,low,unacc


In [261]:
df_car_train.columns.tolist()

['low', 'vhigh', '4', '4.1', 'big', 'med', 'acc']

In [262]:
import pandas as pd

# Define the column names as a list
column_names = ['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety', 'label']
# Read the CSV file with the specified column names
df_car_train = pd.read_csv("Data/car-4/train.csv", names=column_names)

df_car_train.head()

,buying,maint,doors,persons,lug_boot,safety,label
0,low,vhigh,4,4,big,med,acc
1,low,high,5more,4,med,high,vgood
2,vhigh,med,2,2,big,high,unacc
3,high,high,2,2,small,high,unacc
4,vhigh,low,3,2,big,low,unacc


In [264]:
car_attributes = df_car_train.columns.tolist()

In [273]:
# Function of read data
import pandas as pd

def read_data(file_path, column_names):
    # Read the CSV file with the specified column names
    df = pd.read_csv(file_path, names=column_names)
    # Extract the list of attributes from the DataFrame's columns
    attributes = df.columns.tolist()[:-1]
    return df, attributes

# Example usage:
file_path = "Data/car-4/train.csv"
column_names = ['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety', 'label']
df_car_train, car_attributes = read_data(file_path, column_names)


In [274]:
df_car_train.head()

,buying,maint,doors,persons,lug_boot,safety,label
0,low,vhigh,4,4,big,med,acc
1,low,high,5more,4,med,high,vgood
2,vhigh,med,2,2,big,high,unacc
3,high,high,2,2,small,high,unacc
4,vhigh,low,3,2,big,low,unacc


In [275]:
print(car_attributes)

['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety']


In [276]:
test_file_path = "Data/car-4/test.csv"
column_names = ['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety', 'label']
df_car_test, car_attributes = read_data(file_path, column_names)

In [277]:
root = ID3(df_car_train, car_attributes, max_depth=1, purity_measurement='entropy')

Starting implementing ID3 with purity_measurement entropy
class list in this iteration ['good', 'vgood', 'acc', 'unacc']
------Root is none, first time creating root node-------
------starting creating root node-------
starting finding the best feature
data count for input df:  1000
feature list in find best_attribute ['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety']
For buying the information gain is 0.10152470712485662
For maint the information gain is 0.07741985577459665
For doors the information gain is 0.006726514230977809
For persons the information gain is 0.22441128678577127
For lug_boot the information gain is 0.03688725199484155
For safety the information gain is 0.25822501448993573
best attribute:  safety
remaining_attributes:  ['buying', 'maint', 'doors', 'persons', 'lug_boot']
value in attribute values: med
Check current root:  Attribute: safety, Label: None
Child:
  Attribute: med, Label: None

Starting implementing ID3 with purity_measurement entropy
class li

In [278]:
print(root)

Attribute: safety, Label: None
Child:
  Attribute: med, Label: unacc
Child:
  Attribute: high, Label: unacc
Child:
  Attribute: low, Label: unacc



In [279]:
# make predictions
predictions = []
for index, row in df_car_test.iterrows():
    prediction = predict(root, row)  
    predictions.append(prediction)

# Calculate testing error
testing_predictions = predictions
testing_true_labels = df_car_test.iloc[:, -1].tolist()

testing_error_rate = calculate_error_rate(testing_predictions, testing_true_labels)
print("Testing Error Rate:", testing_error_rate) 

Testing Error Rate: 0.302


In [320]:
# Define the column names as a list
column_names = ['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety', 'label']
# Read the CSV file with the specified column names
df_car_train = pd.read_csv("Data/car-4/train.csv", names=column_names)
df_car_test  = pd.read_csv("Data/car-4/test.csv", names=column_names)
car_attributes = column_names[:-1]
print(car_attributes)

['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety']


In [321]:
# run tree and make prediction and cal error
root = ID3(df_car_train, car_attributes, max_depth=1, purity_measurement='entropy')
predictions = make_predictions(root, df_car_test)
true_labels = df_car_test.iloc[:, -1].tolist()
error_rate = calculate_error_rate(predictions, true_labels)

Starting implementing ID3 with purity_measurement entropy
class list in this iteration ['good', 'vgood', 'acc', 'unacc']
------Root is none, first time creating root node-------
------starting creating root node-------
starting finding the best feature
data count for input df:  1000
feature list in find best_attribute ['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety']
For buying the information gain is 0.10152470712485662
For maint the information gain is 0.07741985577459665
For doors the information gain is 0.006726514230977809
For persons the information gain is 0.22441128678577127
For lug_boot the information gain is 0.03688725199484155
For safety the information gain is 0.25822501448993573
best attribute:  safety
remaining_attributes:  ['buying', 'maint', 'doors', 'persons', 'lug_boot']
value in attribute values: med
Check current root:  Attribute: safety, Label: None
Child:
  Attribute: med, Label: None

Starting implementing ID3 with purity_measurement entropy
class li

In [322]:
print(error_rate)

0.2967032967032967


In [324]:
# run tree and make prediction and cal error
root = ID3(df_car_train, car_attributes, max_depth=2, purity_measurement='entropy')
predictions = make_predictions(root, df_car_test)
true_labels = df_car_test.iloc[:, -1].tolist()
error_rate = calculate_error_rate(predictions, true_labels)

Starting implementing ID3 with purity_measurement entropy
class list in this iteration ['good', 'vgood', 'acc', 'unacc']
------Root is none, first time creating root node-------
------starting creating root node-------
starting finding the best feature
data count for input df:  1000
feature list in find best_attribute ['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety']
For buying the information gain is 0.10152470712485662
For maint the information gain is 0.07741985577459665
For doors the information gain is 0.006726514230977809
For persons the information gain is 0.22441128678577127
For lug_boot the information gain is 0.03688725199484155
For safety the information gain is 0.25822501448993573
best attribute:  safety
remaining_attributes:  ['buying', 'maint', 'doors', 'persons', 'lug_boot']
value in attribute values: med
Check current root:  Attribute: safety, Label: None
Child:
  Attribute: med, Label: None

Starting implementing ID3 with purity_measurement entropy
class li

In [325]:
print(error_rate)

0.6607142857142857


In [229]:
# Sample test for TreeNode Class
class TreeNode: 
    def __init__(self, label=None, attributes=None, children=None):
        self.label = label  # value of the node
        self.attributes = attributes
        self.children = children or []  # dict of child nodes
    
    def __str__(self, level=0):
        prefix = "  " * level
        result = prefix + f"Attribute: {self.attributes}, Label: {self.label}\n"
        for child in self.children:
            result += prefix + f"Child:\n"
            result += child.__str__(level + 1)
        return result
    # def __str__(self):
    #     return str(self.value)

    def add_child(self, child_node):
        self.children.append(child_node)

# tree = TreeNode()
# tree.attributes = {}
# tree.attributes = "Outlook"
# child_node = TreeNode()
# child_node.attributes = 'Sunny'

# tree.add_child(child_node)
# print(tree)
import random

tree = TreeNode()
tree.attributes = "Outlook"
print(tree)

Label = ["-", "-"]
unique_labels = set(Label)
print("unique_labels: ", unique_labels)
label_list = list(unique_labels)
random_label = random.choice(label_list)

attribute_values = ['Sunny','Overcast', 'Rain']
for value in attribute_values:
    child_node = TreeNode()
    child_node.attributes = value  
    tree.add_child(child_node) 
    grand_child = TreeNode()
    grand_child.attributes = "Humidity"
    child_node.add_child(grand_child)

    child_node.label = random_label
    tree.add_child(child_node)


print("print tree", tree)

Attribute: Outlook, Label: None

unique_labels:  {'-'}
print tree Attribute: Outlook, Label: None
Child:
  Attribute: Sunny, Label: -
  Child:
    Attribute: Humidity, Label: None
Child:
  Attribute: Sunny, Label: -
  Child:
    Attribute: Humidity, Label: None
Child:
  Attribute: Overcast, Label: -
  Child:
    Attribute: Humidity, Label: None
Child:
  Attribute: Overcast, Label: -
  Child:
    Attribute: Humidity, Label: None
Child:
  Attribute: Rain, Label: -
  Child:
    Attribute: Humidity, Label: None
Child:
  Attribute: Rain, Label: -
  Child:
    Attribute: Humidity, Label: None



In [166]:
df_S = pd.DataFrame(S)

best_attribute = 'Outlook'
value = 'Sunny'
Sv = df_S[df_S[best_attribute] == value]
remaining_label = Sv.iloc[:,-1].tolist()

print(remaining_label)
print(Sv)

['-', '-', '-', '+', '+']
   Outlook Temperature Humidity    Wind Play
0    Sunny         Hot     High    Weak    -
1    Sunny         Hot     High  Strong    -
7    Sunny        Mild     High    Weak    -
8    Sunny        Cool   Normal    Weak    +
10   Sunny        Mild   Normal  Strong    +


In [94]:
root = TreeNode()
root.attributes = "Outlook"
attribute_list = ["Sunny", "Rain"]
for value in attribute_list:
    child_node = TreeNode()
    child_node.attributes = value
    root.add_child(child_node)
print(root)

Attribute: Outlook, Label: None
Child:
  Attribute: Sunny, Label: None
Child:
  Attribute: Rain, Label: None



In [79]:
# Sample Usage
S = [
    {"Outlook": "Sunny", "Temperature": "Hot", "Humidity": "High", "Wind": "Weak", "Play": "-"},
    {"Outlook": "Sunny", "Temperature": "Hot", "Humidity": "High", "Wind": "Strong", "Play": "-"},
    {"Outlook": "Overcast", "Temperature": "Hot", "Humidity": "High", "Wind": "Weak", "Play": "+"},
    {"Outlook": "Rain", "Temperature": "Mild", "Humidity": "High", "Wind": "Weak", "Play": "+"},
    {"Outlook": "Rain", "Temperature": "Cool", "Humidity": "Normal", "Wind": "Weak", "Play": "+"},
    {"Outlook": "Rain", "Temperature": "Cool", "Humidity": "Normal", "Wind": "Strong", "Play": "-"},
    {"Outlook": "Overcast", "Temperature": "Cool", "Humidity": "Normal", "Wind": "Strong", "Play": "+"},
    {"Outlook": "Sunny", "Temperature": "Mild", "Humidity": "High", "Wind": "Weak", "Play": "-"},
    {"Outlook": "Sunny", "Temperature": "Cool", "Humidity": "Normal", "Wind": "Weak", "Play": "+"},
    {"Outlook": "Rain", "Temperature": "Mild", "Humidity": "Normal", "Wind": "Weak", "Play": "+"},
    {"Outlook": "Sunny", "Temperature": "Mild", "Humidity": "Normal", "Wind": "Strong", "Play": "+"},
    {"Outlook": "Overcast", "Temperature": "Mild", "Humidity": "High", "Wind": "Strong", "Play": "+"},
    {"Outlook": "Overcast", "Temperature": "Hot", "Humidity": "Normal", "Wind": "Weak", "Play": "+"},
    {"Outlook": "Rain", "Temperature": "Mild", "Humidity": "High", "Wind": "Weak", "Play": "-"},
]

df_S = pd.DataFrame(S)

# Find most common label
def find_most_common_label(df):
    labels = df.iloc[:, -1].tolist()
    label_counts = count_elements(labels)
    print("label_counts: ", label_counts)
    most_common_label = max(label_counts, key=lambda k: label_counts[k])
    return most_common_label

def count_elements(lst):
    element_counts = {}
    for element in lst:
        if element in element_counts:
            element_counts[element] += 1
        else:
            element_counts[element] = 1
    return element_counts

test_common_label = find_most_common_label(df_S)
print(test_common_label)

# np.bincount(df_S.iloc[:, -1].tolist()).argmax()

label_counts:  {'-': 5, '+': 9}
+


In [102]:
# Sample Usage
S = [
    {"Outlook": "Sunny", "Temperature": "Hot", "Humidity": "High", "Wind": "Weak", "Play": "-"},
    {"Outlook": "Sunny", "Temperature": "Hot", "Humidity": "High", "Wind": "Strong", "Play": "-"},
    {"Outlook": "Overcast", "Temperature": "Hot", "Humidity": "High", "Wind": "Weak", "Play": "+"},
    {"Outlook": "Rain", "Temperature": "Mild", "Humidity": "High", "Wind": "Weak", "Play": "+"},
    {"Outlook": "Rain", "Temperature": "Cool", "Humidity": "Normal", "Wind": "Weak", "Play": "+"},
    {"Outlook": "Rain", "Temperature": "Cool", "Humidity": "Normal", "Wind": "Strong", "Play": "-"},
    {"Outlook": "Overcast", "Temperature": "Cool", "Humidity": "Normal", "Wind": "Strong", "Play": "+"},
    {"Outlook": "Sunny", "Temperature": "Mild", "Humidity": "High", "Wind": "Weak", "Play": "-"},
    {"Outlook": "Sunny", "Temperature": "Cool", "Humidity": "Normal", "Wind": "Weak", "Play": "+"},
    {"Outlook": "Rain", "Temperature": "Mild", "Humidity": "Normal", "Wind": "Weak", "Play": "+"},
    {"Outlook": "Sunny", "Temperature": "Mild", "Humidity": "Normal", "Wind": "Strong", "Play": "+"},
    {"Outlook": "Overcast", "Temperature": "Mild", "Humidity": "High", "Wind": "Strong", "Play": "+"},
    {"Outlook": "Overcast", "Temperature": "Hot", "Humidity": "Normal", "Wind": "Weak", "Play": "+"},
    {"Outlook": "Rain", "Temperature": "Mild", "Humidity": "High", "Wind": "Weak", "Play": "-"},
]

df_S = pd.DataFrame(S)

Attributes = ["Outlook", "Temperature", "Humidity", "Wind"]
Label = ["-", "-", "+", "+", "+", "-", "+", "-", "+", "+", "+", "+", "+", "-"]
Target_var = 'Play'


root = ID3(df_S, Attributes, Label, purity_measurement='entropy')  # You can change purity_measurement as needed

purity_measurement: entropy
starting create root node
starting finding the most informative feature
starting calculating Outlook information gain
starting feature_value=Sunny in the list
starting feature_value=Overcast in the list
starting feature_value=Rain in the list
For Outlook the information gain is 0.24674981977443933
starting calculating Temperature information gain
starting feature_value=Hot in the list
starting feature_value=Mild in the list
starting feature_value=Cool in the list
For Temperature the information gain is 0.02922256565895487
starting calculating Humidity information gain
starting feature_value=High in the list
starting feature_value=Normal in the list
For Humidity the information gain is 0.15183550136234159
starting calculating Wind information gain
starting feature_value=Weak in the list
starting feature_value=Strong in the list
For Wind the information gain is 0.003184853044648994
max info feature:  Outlook
best attribute:  Outlook
attribute_values:  ['Sunny'

In [103]:
print(root)

Attribute: {}, Label: -



In [41]:
root.attributes

{}

In [98]:
# Sample Usage
S = [
    {"Outlook": "Sunny", "Temperature": "Hot", "Humidity": "High", "Wind": "Weak", "Play": "-"},
    {"Outlook": "Sunny", "Temperature": "Hot", "Humidity": "High", "Wind": "Strong", "Play": "-"},
    {"Outlook": "Overcast", "Temperature": "Hot", "Humidity": "High", "Wind": "Weak", "Play": "+"},
    {"Outlook": "Rain", "Temperature": "Mild", "Humidity": "High", "Wind": "Weak", "Play": "+"},
    {"Outlook": "Rain", "Temperature": "Cool", "Humidity": "Normal", "Wind": "Weak", "Play": "+"},
    {"Outlook": "Rain", "Temperature": "Cool", "Humidity": "Normal", "Wind": "Strong", "Play": "-"},
    {"Outlook": "Overcast", "Temperature": "Cool", "Humidity": "Normal", "Wind": "Strong", "Play": "+"},
    {"Outlook": "Sunny", "Temperature": "Mild", "Humidity": "High", "Wind": "Weak", "Play": "-"},
    {"Outlook": "Sunny", "Temperature": "Cool", "Humidity": "Normal", "Wind": "Weak", "Play": "+"},
    {"Outlook": "Rain", "Temperature": "Mild", "Humidity": "Normal", "Wind": "Weak", "Play": "+"},
    {"Outlook": "Sunny", "Temperature": "Mild", "Humidity": "Normal", "Wind": "Strong", "Play": "+"},
    {"Outlook": "Overcast", "Temperature": "Mild", "Humidity": "High", "Wind": "Strong", "Play": "+"},
    {"Outlook": "Overcast", "Temperature": "Hot", "Humidity": "Normal", "Wind": "Weak", "Play": "+"},
    {"Outlook": "Rain", "Temperature": "Mild", "Humidity": "High", "Wind": "Weak", "Play": "-"},
]

df_S = pd.DataFrame(S)

attribute_values = df_S['Outlook'].unique().tolist()
# attribute_values_str = ', '.join(attribute_values)
# print(attribute_values_str)
print(attribute_values)

['Sunny', 'Overcast', 'Rain']
